In [2]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=3a0a1fe78a7168c0fef643648d32d49484d31925d905c3e5ae99abc5aa7b409e
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [3]:
import pandas as pd
import ta  # pip install ta

# 1) Carga y ordena el histórico
hist = pd.read_csv("price_hist_master.csv", parse_dates=["date"])
hist = hist.sort_values(["symbol","date"])

# 2) Calcula indicadores (MA7, RSI14, etc.) — opcional pero recomendado
features = []
for sym, df_sym in hist.groupby("symbol"):
    df_sym = df_sym.copy()
    df_sym["ma7"] = df_sym["price"].rolling(7).mean()
    df_sym["rsi14"] = ta.momentum.RSIIndicator(df_sym["price"],14).rsi()
    # … otras features …
    features.append(df_sym)
hist_feat = pd.concat(features, ignore_index=True)

# 3) Genera los targets por shift
threshold = 0.20
for H in [30,60,90]:
    # price futuro H días adelante
    hist_feat[f"price_future_{H}d"] = hist_feat.groupby("symbol")["price"].shift(-H)
    # retorno
    hist_feat[f"target_return_{H}d"] = hist_feat[f"price_future_{H}d"] / hist_feat["price"] - 1
    # etiqueta binaria
    hist_feat[f"target_up_{H}d"] = (hist_feat[f"target_return_{H}d"] >= threshold).astype(int)

# 4) Limpia filas sin target (las últimas H por moneda)
hist_feat.dropna(subset=[f"target_return_{H}d" for H in [30,60,90]], inplace=True)

# 5) Guarda tu dataset de entrenamiento
hist_feat.to_csv("training_from_history.csv", index=False)
print("✅ training_from_history.csv listo con targets a 30/60/90 días y features históricas.")


✅ training_from_history.csv listo con targets a 30/60/90 días y features históricas.


In [5]:
import pandas as pd

df = pd.read_csv("training_from_history.csv", parse_dates=["date"])

# Ordena por fecha
df = df.sort_values("date")

# Define cuántos días de “test” quieres reservar
test_size = 0.2  # por ejemplo el 20 %

n     = len(df)
split = int(n*(1-test_size))

train_df = df.iloc[:split]
test_df  = df.iloc[split:]

In [6]:
import pandas as pd

# 1) Carga el dataset histórico con targets
df = pd.read_csv("training_from_history.csv", parse_dates=["date"])

# 2) Ordena por fecha ascendente
df = df.sort_values("date")

# 3) Define el tamaño de test (20% final)
test_size = 0.2
n = len(df)
split = int(n * (1 - test_size))

# 4) Crea los datasets de entrenamiento y prueba
train_df = df.iloc[:split].reset_index(drop=True)
test_df  = df.iloc[split:].reset_index(drop=True)

# 5) Guarda como CSV independientes
train_df.to_csv("train_history.csv", index=False)
test_df.to_csv("test_history.csv",  index=False)

print("✅ Se han guardado 'train_history.csv' y 'test_history.csv' con el split 80/20.")


✅ Se han guardado 'train_history.csv' y 'test_history.csv' con el split 80/20.
